In [3]:
import pandas as pd
from transformers import  AlbertConfig, AlbertModel
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from transformers import AlbertTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

In [4]:
# Load training data from "train.csv"
train_df = pd.read_csv("synthetic_train.csv")

# Load validation data from "val.csv"
val_df = pd.read_csv("synthetic_val.csv")

label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['sentiment'])
val_df['label_encoded'] = label_encoder.transform(val_df['sentiment'])


In [5]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True)


In [6]:
train_df['input_ids'] = train_df['sentence'].apply(tokenize_text)
val_df['input_ids'] = val_df['sentence'].apply(tokenize_text)



In [7]:
albert_config = AlbertConfig(
    vocab_size=30000,            # Size of the vocabulary
    hidden_size=512,            # Size of the hidden layers
    num_hidden_layers=10,        # Number of hidden layers
    num_attention_heads=10,      # Number of attention heads
    intermediate_size=1024,     # Size of the intermediate (feed-forward) layers
    hidden_dropout_prob=0.2,    # Dropout probability for hidden layers
    attention_probs_dropout_prob=0.2,  # Dropout probability for attention scores
    max_position_embeddings=128,  # Maximum position embeddings (adjust based on your sequence length)
    type_vocab_size=2,          # Number of token types (typically 0 for regular text, 1 for special tokens)
)



In [8]:
input_layer = Input(shape=(128,), dtype=tf.int32)
embedding_layer = Embedding(input_dim=albert_config.vocab_size, output_dim=albert_config.hidden_size)(input_layer)
pooling_layer = GlobalAveragePooling1D()(embedding_layer)
output_layer = Dense(units=len(label_encoder.classes_), activation='softmax')(pooling_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])


In [9]:
train_input_ids = np.array(train_df['input_ids'].to_list())
val_input_ids = np.array(val_df['input_ids'].to_list())

In [10]:

history = model.fit(
    train_input_ids, train_df['label_encoded'].values,
    validation_data=(val_input_ids, val_df['label_encoded'].values),
    epochs=10, batch_size=128
)

Epoch 1/10
64/64 [==============================] - 27s 404ms/step - loss: 1.0970 - accuracy: 0.4284 - val_loss: 1.0950 - val_accuracy: 0.4479
Epoch 2/10
64/64 [==============================] - 28s 437ms/step - loss: 1.0930 - accuracy: 0.4206 - val_loss: 1.0912 - val_accuracy: 0.4764
Epoch 3/10
64/64 [==============================] - 25s 392ms/step - loss: 1.0889 - accuracy: 0.5259 - val_loss: 1.0872 - val_accuracy: 0.5688
Epoch 4/10
64/64 [==============================] - 25s 388ms/step - loss: 1.0845 - accuracy: 0.5263 - val_loss: 1.0828 - val_accuracy: 0.5580
Epoch 5/10
64/64 [==============================] - 23s 359ms/step - loss: 1.0796 - accuracy: 0.6054 - val_loss: 1.0778 - val_accuracy: 0.6218
Epoch 6/10
64/64 [==============================] - 24s 380ms/step - loss: 1.0742 - accuracy: 0.5906 - val_loss: 1.0722 - val_accuracy: 0.6090
Epoch 7/10
64/64 [==============================] - 24s 382ms/step - loss: 1.0680 - accuracy: 0.6586 - val_loss: 1.0660 - val_accuracy: 0.6341

In [11]:
test_input_ids = np.array(val_df['input_ids'].to_list())
test_loss, test_accuracy = model.evaluate(test_input_ids, val_df['label_encoded'].values)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


64/64 [==============================] - 1s 8ms/step - loss: 1.0434 - accuracy: 0.6488
Test Loss: 1.0434, Test Accuracy: 0.6488


test_loss, test_accuracy = model.evaluate(test_df['input_ids'].to_list(), test_df['label_encoded'].values)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
model.save('sentiment_model')

In [ ]:
sample_text = ""
encoded_text = tokenize_text(sample_text)
encoded_text = np.array([encoded_text]) 
predicted_class = model.predict(encoded_text)[0]
predicted_sentiment = label_encoder.inverse_transform([predicted_class.argmax()])[0]
print(f"Predicted Sentiment: {predicted_sentiment}")